Baixar o arquivo já analisado.
[Pré-treinado](https://pjreddie.com/media/files/yolov3.weights)

# YOLO - You Only Look Once
É um sistema de detecção de objetos em tempo real de última geração.

In [ ]:
import cv2
import numpy as np
import time

yolov3.cfg: Arquivo de configuração do yolo (default) utilizado para setar a saturação, exposição e canais de cores para ser analisados.

yolov3.weigths: Arquivo pré treinado.

coco: Arquivo com as respectivas label's.

In [ ]:
#Carrega o YOLO

net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []

with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()] #pega os nomes dentro do arquivo.

In [ ]:
#Separação das camadas
layer_names = net.getLayerNames()

outputlayers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0,255, size=(len(classes), 3))

In [ ]:
video = cv2.VideoCapture('loja.mp4')
writer = None #Utilizado para geração do video de saída
fonte = cv2.FONT_HERSHEY_PLAIN
inicio = time.time()
frame_id = 0
count = 0
writer = None
(W, H) = (None, None)
try:
    prop = cv2.CAP_PROP_FRAME_COUNT
    total = int(video.get(prop))
    print("[INFO] {} total de frames no video".format(total))
except:
    printf("Erro, não foi possível determinar a quantidade de frames no video") 


In [ ]:
while True:
    (ret, frame) = video.read()
    if not ret:
        break
    if W is None or H is None:
        (H,W) = frame.shape[:2]

    height,width,channels = frame.shape #Propriedades do frame analisado

    #Converte a imagem para blob para utilizar com o openCV
    blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB = True, crop = False)

    #retangulo de área para ser identificado
    y_inicio = 250
    x_inicio = 700
    cv2.rectangle(frame, (20,y_inicio), (x_inicio,x_inicio), (0,255,0), thickness = 2)
    net.setInput(blob)
    outs = net.forward(outputlayers)
    #print(outs[1])


    #Pega o nível de confiança de um objeto no BLOB analisado
    class_ids=[]
    confidences=[]
    boxes=[]
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confianca = scores[class_id]
            #Nível de confiança desejado
            if confianca > 0.8:
                #Objeto detectado
                center_x= int(detection[0]*width)
                center_y= int(detection[1]*height)

                w = int(detection[2]*width)
                h = int(detection[3]*height)

                x=int(center_x - w/2)
                y=int(center_y - h/2)

                boxes.append([x,y,w,h]) #Insere o retangulo de detecção pelo yolo no vetor 
                confidences.append(float(confianca)) #Insere o nível de confiança detectado
                class_ids.append(class_id) #Nome do objeto detectado
                if writer is None:
                    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
                    writer = cv2.VideoWriter('output_loja.avi', fourcc, 30, (frame.shape[1], frame.shape[0]), True)

    indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)

    for i in range(len(boxes)):
        if i in indexes:
            #Delimitação da região para validação de intruso..
            x,y,w,h = boxes[i]
            if x <= x_inicio and y >= y_inicio:
                #"Intruso" Detectado
               cv2.putText(frame,"INTRUSO",(x,y+80),fonte,2,(0,0,255),2)    
            label = str(classes[class_ids[i]])
            confianca= confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
            cv2.putText(frame,label+" "+str(round(confianca,2)),(x,y+30),fonte,1,(255,255,255),2)

    if writer is not None:
        writer.write(frame)
        print("Escrevendo frame" , count+1)
        count = count + 1

    elapsed_time = time.time() - inicio
    fps=frame_id/elapsed_time
    cv2.putText(frame,"FPS:"+str(round(fps,2)),(10,50),fonte,2,(0,0,0),1)
    
    #cv2.imshow("Image",frame)
    #key = cv2.waitKey(1) #espera 1ms para processar cada frame
    
    #if key == 27: #esc 
     #   break;
print(elapsed_time)
writer.release()
video.release()        
    